# Test02: test code for operations (linear,relu)


# Preparation 

## Preparation of google drive (if required)

In [ ]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更して下さい。

In [ ]:
%cd "ctorch211122/MiniCTorch_Prototype/notebook"

/content/drive/My Drive/Colab Notebooks/ctorch211122/MiniCTorch_Prototype/notebook


## Installation of MiniCTorch

In [1]:
! pip install lark-parser

     |████████████████████████████████| 103 kB 10.7 MB/s 


In [2]:
! pip install git+https://github.com/kojima-r/MiniCTorch_Prototype.git

  Cloning https://github.com/kojima-r/MiniCTorch_Prototype.git to /tmp/pip-req-build-7qb_jvu6
  Running command git clone -q https://github.com/kojima-r/MiniCTorch_Prototype.git /tmp/pip-req-build-7qb_jvu6
  Created wheel for minictorch: filename=minictorch-0.0.1-py3-none-any.whl size=16013 sha256=f33bd3448954b186ffbca7356c2f7a4b886a532dca8bdb2b168fc9544660ee27
  Stored in directory: /tmp/pip-ephem-wheel-cache-clpogahs/wheels/1c/0a/b1/3618887a2db533ac9c7b971dc10589e3976893cdf9a74ca6da
Successfully built minictorch


## ニューラルネットの定義

In [3]:
import sys
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch.generator as GN
import minictorch.converter as CV

In [4]:
class Net( nn.Module ):
   def __init__(self):
      super(Net, self).__init__()
      self.fc1 = nn.Linear(2, 32)
      self.fc2 = nn.Linear(32, 32)
      self.fc3 = nn.Linear(32, 3)

   def forward(self, x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x
      #return F.log_softmax(x, dim = 1)

プロジェクト名、作業フォルダ、JSONファイル名の設定

In [5]:
project = "test02"
folder = "./test"
json_path = folder + '/' + project +'.json'

In [6]:
! mkdir -p test

## 計算グラフをJSONファイルに出力する

In [7]:
def generate_json( json_path, input_x ): 
    model = Net()
    print(input_x)

    model.eval()
    with torch.no_grad():
        print("[JSON]", json_path )
        GN.generate_minictorch_file( model, input_to_model, json_path )
    return model

In [8]:
torch.manual_seed( 1 )
input_to_model = torch.randn((1,2),requires_grad=True)

model = generate_json( json_path, input_to_model )

tensor([[0.6614, 0.2669]], requires_grad=True)
[JSON] ./test/test02.json
skip: Net/Linear[fc1]/weight/fc1
skip: Net/Linear[fc1]/weight/fc1
skip: Net/Linear[fc2]/weight/fc2
skip: Net/Linear[fc2]/weight/fc2
skip: Net/Linear[fc3]/weight/fc3
skip: Net/Linear[fc3]/weight/fc3


JSONファイルをC++コードに変換する

In [9]:
CV.convert_cpp_code( project, folder, model, input_to_model, json_path )

[JSON] ./test/test02.json
{'name': 'Net/Linear[fc1]/weight/weight.7', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [3], 'sorted_id': 1}
{'name': 'Net/Linear[fc1]/bias/bias.7', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [3], 'sorted_id': 2}
{'name': 'Net/Linear[fc2]/weight/weight.9', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [7], 'sorted_id': 5}
{'name': 'Net/Linear[fc2]/bias/bias.9', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [7], 'sorted_id': 6}
{'name': 'Net/Linear[fc3]/weight/weight', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [11], 'sorted_id': 9}
{'name': 'Net/Linear[fc3]/bias/bias', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [11], 'sorted_id': 10}
[PARAM] ./test/test02_param.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [1, 2], 'out': [3], 'sorted_id': 0}
{'name': 'Net/Linear[fc1]/weight/weight.7',

pythonコードを実行する

In [11]:
# check code
model.train()
yy = model( input_to_model )
print(yy)
yy.sum().backward()
print(input_to_model.grad)

tensor([[ 0.1536, -0.1925,  0.0064]], grad_fn=<AddmmBackward0>)
tensor([[-0.2567,  0.2242]])


## 環境構築
(注意) インストール済みの場合は各自の環境に合わせて変更して下さい。

In [12]:
! git clone https://github.com/xtensor-stack/xtensor.git
! git clone https://github.com/xtensor-stack/xtensor-blas.git
! git clone https://github.com/xtensor-stack/xtl.git
! git clone https://github.com/nlohmann/json.git


Cloning into 'xtensor'...
remote: Enumerating objects: 20418, done.
remote: Counting objects: 100% (3446/3446), done.
remote: Compressing objects: 100% (510/510), done.
remote: Total 20418 (delta 3000), reused 3190 (delta 2921), pack-reused 16972
Receiving objects: 100% (20418/20418), 10.08 MiB | 20.36 MiB/s, done.
Resolving deltas: 100% (14570/14570), done.
Cloning into 'xtensor-blas'...
remote: Enumerating objects: 3857, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 3857 (delta 83), reused 96 (delta 41), pack-reused 3688
Receiving objects: 100% (3857/3857), 4.13 MiB | 17.90 MiB/s, done.
Resolving deltas: 100% (3155/3155), done.
Cloning into 'xtl'...
remote: Enumerating objects: 2399, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 2399 (delta 125), reused 137 (delta 58), pack-reused 2141
Receiving objects: 100% (2399/2399), 722.73 KiB | 7.94 

In [13]:
! git clone https://github.com/kojima-r/MiniCTorch_Prototype.git

Cloning into 'MiniCTorch_Prototype'...
remote: Enumerating objects: 728, done.
remote: Counting objects: 100% (728/728), done.
remote: Compressing objects: 100% (359/359), done.
remote: Total 728 (delta 499), reused 583 (delta 367), pack-reused 0
Receiving objects: 100% (728/728), 42.78 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (499/499), done.


C++コードをコンパイル、リンクする

In [14]:
C_SRC = './test/test02.cpp ./test/test02_param.cpp'
C_OPT = '-I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include '
C_OUT = './test/test02'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test02.cpp ./test/test02_param.cpp
OPT = -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test02


In [15]:
! ./test/test02

### forward computation ...
{{ 0.153593, -0.192527,  0.006398}}
### backward computation ...
input_grad{{-0.128356,  0.112093}}
